# Grouping

Chapter 9 in Python for Data Analysis demonstrates a variety of methods to analyze data via data aggregation and grouping operations. Those are the focus of this session.  Our overall goal for this session is to do Exploratory Data Analysis, which is essentially looking at and probing our data to learn about the patterns we can discover in them.  Often this can generate a better understanding of problems in the data, as well as revealing relationships that might be worth exploring in more depth.


For this exercise, we will use 2010 US Census data, for all census tracts in Kentucky.  A good starting point for any kind of Census data is the American Fact Finder: 

https://factfinder.census.gov/faces/nav/jsf/pages/index.xhtml

The Census Bureau produces several products, the most famous of which is the decennial census, which as its name implies, is a true Census.  There are a limited number of data fields for the whole population (or at least those that the Census workers are able to reach, which is pretty good).  The Census has other products that are surveys, such as the American Community Survey (ACS) which asks more detailed questions on an annual basis from a 1% sample of households.  

In this case, we are working with file DP-1: Profile of General Population and Housing Characteristics: 2010, which is also a part of something called Summary File 1 (SF-1).  It is for the 100% sample.  A data dictionary for selected fields is below.  

In [1]:
import pandas as pd

# skip the second row, which contains descriptions
sf1 = pd.read_csv('data/DEC_10_SF1_combined.csv', skiprows=[1])

# make the tract, county and state separate
sf1['tract'] = sf1['GEO.display-label'].apply(lambda x: x.split(sep=',')[0])
sf1['county'] = sf1['GEO.display-label'].apply(lambda x: x.split(sep=',')[1])
sf1['state'] = sf1['GEO.display-label'].apply(lambda x: x.split(sep=',')[2])

# compute some derived fields
sf1['pct_rent'] = sf1['H4-D004'] / sf1['H4-D001'] * 100
sf1['pct_black'] = sf1['P3-D003'] / sf1['P3-D001'] * 100
sf1['pct_asian'] = sf1['P3-D005'] / sf1['P3-D001'] * 100
sf1['pct_white'] = sf1['P3-D002'] / sf1['P3-D001'] * 100
sf1['pct_hisp'] = sf1['P4-D003'] / sf1['P4-D001'] * 100
sf1['pct_vacant'] = sf1['H5-D001'] / sf1['H1-D001'] * 100
sf1[:5]

,GEO.id,GEO.id2,GEO.display-label,P1-D001,P3-D001,P3-D002,P3-D003,P3-D004,P3-D005,P3-D006,...,H5-D008,tract,county,state,pct_rent,pct_black,pct_asian,pct_white,pct_hisp,pct_vacant
0,1400000US21001970100,21001970100,"Census Tract 9701, Adair County, Kentucky",1727,1727,1683,14,1,0,1,...,60,Census Tract 9701,Adair County,Kentucky,17.411402,0.810654,0.000000,97.452229,2.142444,16.794872
1,1400000US21001970200,21001970200,"Census Tract 9702, Adair County, Kentucky",1722,1722,1635,41,5,0,0,...,35,Census Tract 9702,Adair County,Kentucky,19.252874,2.380952,0.000000,94.947735,2.613240,25.161290
2,1400000US21001970300,21001970300,"Census Tract 9703, Adair County, Kentucky",3016,3016,2944,6,11,8,0,...,106,Census Tract 9703,Adair County,Kentucky,20.521173,0.198939,0.265252,97.612732,1.856764,15.193370
3,1400000US21001970401,21001970401,"Census Tract 9704.01, Adair County, Kentucky",4070,4070,3716,237,1,16,1,...,109,Census Tract 9704.01,Adair County,Kentucky,37.215909,5.823096,0.393120,91.302211,1.547912,10.795743
4,1400000US21001970402,21001970402,"Census Tract 9704.02, Adair County, Kentucky",4261,4261,3950,180,16,16,3,...,70,Census Tract 9704.02,Adair County,Kentucky,30.911681,4.224360,0.375499,92.701244,1.900962,11.642542


## Groupby and Aggregation Operations

Groupby is a powerful method in pandas that follows the split-apply-combine approach to data.  As shown in Figure 9-1 in the context of a sum operation, the data is first split into groups that share the same key values.  Then an operation, in this case a sum, is applied to each group.  Then the results are combined.

The built-in aggregation methods available for groupby operations include:
* count
* sum
* mean
* median
* std, var
* min, max
* first, last

You can also apply your own functions as aggregation methods.

![Groupby Operations](groupby.png "Groupby")

Let's apply this approach to computing total population in each county in our dataset.  We can do this in two steps to help explain what is happening.  First we create a groupby object, using county codes to group all the census blocks in sf1 into groups that share the same county code.

In [3]:
grouped = sf1['P1-D001'].groupby(sf1['county'])
grouped

Now that we have this grouping object that represents the **split** part of the workflow in the figure above, we can **apply** operations and **combine** the results using methods like sum:

In [4]:
grouped.sum()

county
 Adair County            18656
 Allen County            19956
 Anderson County         21421
 Ballard County           8249
 Barren County           42173
 Bath County             11591
 Bell County             28691
 Boone County           118811
 Bourbon County          19985
 Boyd County             49542
 Boyle County            28432
 Bracken County           8488
 Breathitt County        13878
 Breckinridge County     20059
 Bullitt County          74319
 Butler County           12690
 Caldwell County         12984
 Calloway County         37191
 Campbell County         90336
 Carlisle County          5104
 Carroll County          10811
 Carter County           27720
 Casey County            15955
 Christian County        73955
 Clark County            35613
 Clay County             21730
 Clinton County          10272
 Crittenden County        9315
 Cumberland County        6856
 Daviess County          96656
                         ...  
 Nicholas County          7135
 

We might want to capture the result in a DataFrame if we want to use it in other processing, like merging the results to the original DataFrame.

In [5]:
county_pop = sf1['P1-D001'].groupby(sf1['county']).sum().to_frame(name='county_population')
county_pop

,county_population
county,
Adair County,18656
Allen County,19956
Anderson County,21421
Ballard County,8249
Barren County,42173
Bath County,11591
Bell County,28691
Boone County,118811
Bourbon County,19985


Here we merge the county total population with sf1 and create a new DataFrame.

In [6]:
sf2 = pd.merge(sf1,county_pop, left_on='county', right_index=True)
sf2[:5]

,GEO.id,GEO.id2,GEO.display-label,P1-D001,P3-D001,P3-D002,P3-D003,P3-D004,P3-D005,P3-D006,...,tract,county,state,pct_rent,pct_black,pct_asian,pct_white,pct_hisp,pct_vacant,county_population
0,1400000US21001970100,21001970100,"Census Tract 9701, Adair County, Kentucky",1727,1727,1683,14,1,0,1,...,Census Tract 9701,Adair County,Kentucky,17.411402,0.810654,0.000000,97.452229,2.142444,16.794872,18656
1,1400000US21001970200,21001970200,"Census Tract 9702, Adair County, Kentucky",1722,1722,1635,41,5,0,0,...,Census Tract 9702,Adair County,Kentucky,19.252874,2.380952,0.000000,94.947735,2.613240,25.161290,18656
2,1400000US21001970300,21001970300,"Census Tract 9703, Adair County, Kentucky",3016,3016,2944,6,11,8,0,...,Census Tract 9703,Adair County,Kentucky,20.521173,0.198939,0.265252,97.612732,1.856764,15.193370,18656
3,1400000US21001970401,21001970401,"Census Tract 9704.01, Adair County, Kentucky",4070,4070,3716,237,1,16,1,...,Census Tract 9704.01,Adair County,Kentucky,37.215909,5.823096,0.393120,91.302211,1.547912,10.795743,18656
4,1400000US21001970402,21001970402,"Census Tract 9704.02, Adair County, Kentucky",4261,4261,3950,180,16,16,3,...,Census Tract 9704.02,Adair County,Kentucky,30.911681,4.224360,0.375499,92.701244,1.900962,11.642542,18656


## Transforming Data with Groupby

In some cases you may want to apply a function to your data, by group.  An example would be to normalize a column by a mean of each group.  Say we wanted to subtract the mean population density of each county from the population density of each census block. We could write a function to subtract the mean from each value, and then use the transform operation to apply this to each group:

In [7]:
def demean(arr):
    return arr - arr.mean()

Now we can apply this tranformation to columns in our dataframe.  As examples, let's 'demean' the pct_black and pct_rent columns, subtracting the county-wide mean of these values from the tract-specific values, so that the result is transformed to have a mean of zero within each county.

To check the results, we print the means per county, then the original values for the first 5 rows, then the transformed results.  The transformed results we should be able to calculate by subtracting the appropriate county mean from the tract value.

In [8]:
normalized = sf1[['pct_black', 'pct_rent']].groupby(sf1['county']).transform(demean)
print(sf1[['pct_black', 'pct_rent']].groupby(sf1['county']).mean())
print(sf1[['county','pct_black', 'pct_rent']][:5])
print(normalized[:5])

                      pct_black   pct_rent
county                                    
 Adair County          2.104384  22.886621
 Allen County          0.716792  22.312658
 Anderson County       1.419188  20.908477
 Ballard County        2.690326  21.350937
 Barren County         3.385357  29.982273
 Bath County           1.719977  23.865098
 Bell County           2.017470  32.080953
 Boone County          2.362180  24.779530
 Bourbon County        4.863221  33.604963
 Boyd County           2.690429  33.766804
 Boyle County          7.501400  33.987453
 Bracken County        0.319829  24.755274
 Breathitt County      0.291431  20.408091
 Breckinridge County   1.492590  18.524204
 Bullitt County        0.582663  18.852640
 Butler County         0.293354  20.702463
 Caldwell County       4.372987  22.422207
 Calloway County       4.146136  37.944503
 Campbell County       3.106286  34.861471
 Carlisle County       0.752688  17.877812
 Carroll County        1.184510  31.659932
 Carter Cou

We can merge these transformed results on to the original DataFrame, and check the means of the original variables and the tranformed ones.  The transformed ones should be arbitrarily close to zero.

In [48]:
sf2 = pd.merge(sf1,normalized, left_index=True, right_index=True)

sf2.groupby('county')[['pct_black_x', 'pct_black_y', 'pct_rent_x', 'pct_rent_y']].mean()

,pct_black_x,pct_black_y,pct_rent_x,pct_rent_y
county,,,,
Adair County,2.104384,-2.854859e-16,22.886621,-2.030122e-15
Allen County,0.716792,-5.551115e-17,22.312658,-3.256654e-15
Anderson County,1.419188,8.881784e-17,20.908477,7.105427e-16
Ballard County,2.690326,1.480297e-16,21.350937,0.000000e+00
Barren County,3.385357,0.000000e+00,29.982273,4.618528e-15
Bath County,1.719977,0.000000e+00,23.865098,-1.184238e-15
Bell County,2.017470,4.687608e-16,32.080953,-3.947460e-15
Boone County,2.362180,3.595008e-16,24.779530,-1.184238e-15
Bourbon County,4.863221,1.480297e-16,33.604963,0.000000e+00


Apply is a method we have learned previously, which allows us to apply a function to each row in a DataFrame.  We can also combine apply with groupby to apply functions based on group membership.  For example, the function 'top' sorts an array and selects the top n rows from it.  We provide some defaults for the arguments of how many rows, and the column to use for the selection:

In [49]:
def top(df, n=5, column='pct_rent'):
    return df.sort_values(by=column, ascending=False).head(n)

Using this on the full dataset and setting the number of rows and the column to get the top values for, in this case using pct_rent to override the default argument, we get the top 10 tracts in the region in terms of percentage rental.

In [50]:
top(sf1, n=10, column='pct_rent')

,GEO.id,GEO.id2,GEO.display-label,P1-D001,P3-D001,P3-D002,P3-D003,P3-D004,P3-D005,P3-D006,...,H5-D008,tract,county,state,pct_rent,pct_black,pct_asian,pct_white,pct_hisp,pct_vacant
255,1400000US21061980100,21061980100,"Census Tract 9801, Edmonson County, Kentucky",8,8,8,0,0,0,0,...,0,Census Tract 9801,Edmonson County,Kentucky,100.000000,0.000000,0.000000,100.000000,0.000000,0.000000
508,1400000US21111003000,21111003000,"Census Tract 30, Jefferson County, Kentucky",3565,3565,258,3187,15,1,16,...,7,Census Tract 30,Jefferson County,Kentucky,99.768697,89.396914,0.028050,7.237027,1.290323,7.883523
200,1400000US21047201501,21047201501,"Census Tract 2015.01, Christian County, Kentucky",5315,5315,3580,982,64,47,81,...,8,Census Tract 2015.01,Christian County,Kentucky,99.698568,18.476011,0.884290,67.356538,14.995296,7.072829
415,1400000US21093000202,21093000202,"Census Tract 2.02, Hardin County, Kentucky",4514,4514,3053,863,45,78,20,...,21,Census Tract 2.02,Hardin County,Kentucky,99.679230,19.118299,1.727957,67.634027,14.842712,5.957768
872,1400000US21163970200,21163970200,"Census Tract 9702, Meade County, Kentucky",2683,2683,1916,414,41,30,15,...,161,Census Tract 9702,Meade County,Kentucky,99.270960,15.430488,1.118151,71.412598,15.803205,32.980456
201,1400000US21047201502,21047201502,"Census Tract 2015.02, Christian County, Kentucky",4527,4527,3055,740,45,61,84,...,6,Census Tract 2015.02,Christian County,Kentucky,98.896435,16.346366,1.347471,67.483985,18.003093,6.507937
414,1400000US21093000201,21093000201,"Census Tract 2.01, Hardin County, Kentucky",1409,1409,1150,153,9,19,6,...,60,Census Tract 2.01,Hardin County,Kentucky,98.620690,10.858765,1.348474,81.618169,10.432931,28.571429
833,1400000US21151010500,21151010500,"Census Tract 105, Madison County, Kentucky",4182,4182,3699,330,11,56,6,...,7,Census Tract 105,Madison County,Kentucky,96.078431,7.890961,1.339072,88.450502,1.673840,8.602151
270,1400000US21067000801,21067000801,"Census Tract 8.01, Fayette County, Kentucky",4094,4094,3255,385,6,356,2,...,13,Census Tract 8.01,Fayette County,Kentucky,95.617530,9.404006,8.695652,79.506595,2.002931,36.775819
142,1400000US21037050100,21037050100,"Census Tract 501, Campbell County, Kentucky",1733,1733,1197,368,1,43,0,...,11,Census Tract 501,Campbell County,Kentucky,94.527363,21.234853,2.481246,69.070975,3.693018,7.903780


Below we apply this with groupby and use the defaults for n and column, and it applies the function within each county and concatenates the results, producing the top 5 blocks on pop_sqmi for each county in the region.

In [ ]:
sf1.groupby('county').apply(top)

Here we pass arguments to the function to set n and the column to select the top value from.

In [72]:
sf1.groupby('county').apply(top, n=1, column='P1-D001')

,,GEO.id,GEO.id2,GEO.display-label,P1-D001,P3-D001,P3-D002,P3-D003,P3-D004,P3-D005,P3-D006,...,H5-D008,tract,county,state,pct_rent,pct_black,pct_asian,pct_white,pct_hisp,pct_vacant
county,,,,,,,,,,,,,,,,,,,,,,
Adair County,4,1400000US21001970402,21001970402,"Census Tract 9704.02, Adair County, Kentucky",4261,4261,3950,180,16,16,3,...,70,Census Tract 9704.02,Adair County,Kentucky,30.911681,4.224360,0.375499,92.701244,1.900962,11.642542
Allen County,9,1400000US21003920300,21003920300,"Census Tract 9203, Allen County, Kentucky",4685,4685,4473,83,19,12,0,...,86,Census Tract 9203,Allen County,Kentucky,34.676186,1.771612,0.256137,95.474920,1.173959,9.254975
Anderson County,13,1400000US21005950100,21005950100,"Census Tract 9501, Anderson County, Kentucky",8164,8164,7647,292,5,38,3,...,103,Census Tract 9501,Anderson County,Kentucky,26.186684,3.576678,0.465458,93.667320,1.641352,8.081991
Ballard County,18,1400000US21007950100,21007950100,"Census Tract 9501, Ballard County, Kentucky",4259,4259,4024,145,8,11,0,...,132,Census Tract 9501,Ballard County,Kentucky,22.140011,3.404555,0.258277,94.482273,0.986147,11.664153
Barren County,26,1400000US21009950600,21009950600,"Census Tract 9506, Barren County, Kentucky",5937,5937,5331,299,11,40,10,...,95,Census Tract 9506,Barren County,Kentucky,44.142343,5.036214,0.673741,89.792825,2.779181,11.500354
Bath County,32,1400000US21011970200,21011970200,"Census Tract 9702, Bath County, Kentucky",6192,6192,5960,81,20,10,1,...,202,Census Tract 9702,Bath County,Kentucky,26.202119,1.308140,0.161499,96.253230,1.679587,15.115877
Bell County,35,1400000US21013960200,21013960200,"Census Tract 9602, Bell County, Kentucky",5150,5150,5025,69,8,7,2,...,163,Census Tract 9602,Bell County,Kentucky,34.349030,1.339806,0.135922,97.572816,0.485437,10.937500
Boone County,47,1400000US21015070307,21015070307,"Census Tract 703.07, Boone County, Kentucky",11018,11018,10057,336,25,361,1,...,64,Census Tract 703.07,Boone County,Kentucky,11.996843,3.049555,3.276457,91.277909,1.896896,4.617315
Bourbon County,66,1400000US21017030200,21017030200,"Census Tract 302, Bourbon County, Kentucky",6018,6018,5136,432,17,43,0,...,73,Census Tract 302,Bourbon County,Kentucky,44.381107,7.178465,0.714523,85.343968,6.430708,6.934445


## Exploring Correlations in the Data

Pandas provides simple ways of computing correlation coefficients among the columns in your DataFrame.  If you use corr() on a full DF, it will produce a large correlation table.  A bit hard to navigate and you mostly would not be interested in some of these results.

In [6]:
sf1.corr()

,GEO.id2,P1-D001,P3-D001,P3-D002,P3-D003,P3-D004,P3-D005,P3-D006,P3-D007,P3-D008,...,H5-D005,H5-D006,H5-D007,H5-D008,pct_rent,pct_black,pct_asian,pct_white,pct_hisp,pct_vacant
GEO.id2,1.000000,0.049408,0.049408,0.063873,-0.024147,0.001960,-0.048639,-0.031018,-0.020234,-0.045010,...,0.077670,0.076772,0.010594,0.119563,-0.061820,-0.033330,-0.058690,0.045403,-0.022668,0.043711
P1-D001,0.049408,1.000000,1.000000,0.929708,0.150379,0.479878,0.324468,0.192667,0.240640,0.581058,...,0.308361,0.012925,-0.023479,0.202435,-0.062520,-0.070055,0.136213,0.043546,0.058258,-0.300937
P3-D001,0.049408,1.000000,1.000000,0.929708,0.150379,0.479878,0.324468,0.192667,0.240640,0.581058,...,0.308361,0.012925,-0.023479,0.202435,-0.062520,-0.070055,0.136213,0.043546,0.058258,-0.300937
P3-D002,0.063873,0.929708,0.929708,1.000000,-0.213918,0.363714,0.209791,0.095243,0.057796,0.351332,...,0.376236,0.064984,0.014118,0.236156,-0.257852,-0.391277,0.031737,0.382284,-0.115163,-0.289514
P3-D003,-0.024147,0.150379,0.150379,-0.213918,1.000000,0.252784,0.158195,0.171271,0.325919,0.497941,...,-0.145718,-0.120118,-0.089171,-0.003757,0.464467,0.899663,0.132016,-0.899986,0.302933,0.011119
P3-D004,0.001960,0.479878,0.479878,0.363714,0.252784,1.000000,0.131467,0.463373,0.299747,0.565977,...,0.057503,0.000613,-0.035805,0.090925,0.184507,0.114453,0.060946,-0.170683,0.299351,-0.015777
P3-D005,-0.048639,0.324468,0.324468,0.209791,0.158195,0.131467,1.000000,0.176404,0.160199,0.399051,...,-0.082495,-0.075145,-0.105767,-0.302912,0.163496,0.054982,0.923845,-0.172315,0.178430,-0.222993
P3-D006,-0.031018,0.192667,0.192667,0.095243,0.171271,0.463373,0.176404,1.000000,0.281407,0.523912,...,-0.069520,-0.049845,-0.029479,-0.106864,0.255751,0.099153,0.155861,-0.174929,0.339602,-0.039314
P3-D007,-0.020234,0.240640,0.240640,0.057796,0.325919,0.299747,0.160199,0.281407,1.000000,0.504998,...,-0.108742,-0.086525,-0.017210,-0.124005,0.305578,0.192686,0.134676,-0.334419,0.843366,-0.086896
P3-D008,-0.045010,0.581058,0.581058,0.351332,0.497941,0.565977,0.399051,0.523912,0.504998,1.000000,...,-0.029011,-0.118407,-0.088460,-0.130512,0.425299,0.307009,0.306124,-0.413328,0.466302,-0.216524


It is easy to compute correlation coeffients for a subset of columns.

In [ ]:
sf1[['pct_rent', 'pct_vacant']].corr()

And this method can be combined with groupby to compute correlation tables by group.

In [ ]:
sf1.groupby('county')[['pct_rent', 'pct_vacant']].corr()

## Your turn to practice:

Count the number of census blocks per county.

Calculate total households per county.

Calculate percent renters by county. (Careful not to calculate the mean percent rental across blocks in a county)

Calculate percent vacant by county.

Calculate mean, min and max vacancy rate (at the block level) by county.

Calculate the 90th percentile of vacancy rate (at the block level) by county.

In [61]:
#Count the number of census blocks per county.
sf1['county'].value_counts()

 Jefferson County     191
 Fayette County        82
 Kenton County         41
 Campbell County       25
 Warren County         24
 Daviess County        23
 Hardin County         22
 Boone County          22
 Pike County           19
 Madison County        19
 Christian County      19
 Bullitt County        18
 McCracken County      17
 Pulaski County        14
 Scott County          14
 Oldham County         14
 Laurel County         13
 Boyd County           13
 Hopkins County        12
 Franklin County       11
 Henderson County      11
 Harlan County         11
 Clark County          10
 Floyd County          10
 Barren County         10
 Calloway County        9
 Graves County          9
 Bell County            9
 Greenup County         9
 Jessamine County       9
                     ... 
 McLean County          3
 Leslie County          3
 Bath County            3
 Carroll County         3
 Washington County      3
 Owen County            3
 Ballard County         3
 Lyon County

In [63]:
#Calculate total households per county.
total_households = sf1['H3-D001'].groupby(sf1['county']).sum().to_frame(name='total_households')
total_households


,total_households
county,
Adair County,8568
Allen County,9307
Anderson County,9127
Ballard County,3885
Barren County,19188
Bath County,5405
Bell County,13154
Boone County,46154
Bourbon County,8927


In [64]:
#Calculate percent renters by county.
renters_per_county = sf1['H4-D004'].groupby(sf1['county']).sum().to_frame(name='renters_per_county')
household_per_county = sf1['H4-D001'].groupby(sf1['county']).sum().to_frame(name='household_per_county')

In [66]:
percent_renters = (renters_per_county['renters_per_county'] / household_per_county['household_per_county'] * 100).to_frame(name='percent renters by county')
percent_renters

,percent renters by county
county,
Adair County,25.531915
Allen County,24.184506
Anderson County,24.698291
Ballard County,21.401236
Barren County,32.054827
Bath County,24.198823
Bell County,34.453211
Boone County,25.566920
Bourbon County,36.409228


In [70]:
#4-percent vacant by county
vacant = sf1['H3-D003'].groupby(sf1['county']).sum().to_frame(name='vacant')
Total_unit = sf1['H3-D001'].groupby(sf1['county']).sum().to_frame(name='total')
percent_vacant = (vacant['vacant'] / Total_unit['total'] * 100).to_frame(name='percent vacant')
percent_vacant

,percent vacant
county,
Adair County,14.974323
Allen County,15.676373
Anderson County,8.305029
Ballard County,12.561133
Barren County,11.408172
Bath County,15.134135
Bell County,10.392276
Boone County,6.365645
Bourbon County,10.653075


In [79]:
#Calculate mean, min and max vacancy rate (at the block level) by county.
#Calculate the 90th percentile of vacancy rate (at the block level) by county.
rate_of_vacancy = sf1['pct_vacant'].groupby(sf1['county'])
rate_of_vacancy.max()

county
 Adair County            25.161290
 Allen County            27.559055
 Anderson County         13.452028
 Ballard County          14.648603
 Barren County           22.975207
 Bath County             15.959953
 Bell County             14.950980
 Boone County            12.572614
 Bourbon County          15.599639
 Boyd County             20.585267
 Boyle County            16.060337
 Bracken County          14.001077
 Breathitt County        14.304291
 Breckinridge County     51.259774
 Bullitt County          10.383944
 Butler County           22.043011
 Caldwell County         16.815920
 Calloway County         28.449096
 Campbell County         17.694805
 Carlisle County         14.794816
 Carroll County          17.506631
 Carter County           17.096336
 Casey County            19.819820
 Christian County       100.000000
 Clark County            13.402062
 Clay County             22.991968
 Clinton County          20.645161
 Crittenden County       24.332344
 Cumberland C

In [80]:
rate_of_vacancy = sf1['pct_vacant'].groupby(sf1['county'])
rate_of_vacancy.min()

county
 Adair County           10.795743
 Allen County            9.254975
 Anderson County         5.492297
 Ballard County         11.608392
 Barren County           7.457767
 Bath County            13.480392
 Bell County             7.796194
 Boone County            2.615519
 Bourbon County          6.934445
 Boyd County             6.034483
 Boyle County            8.444444
 Bracken County         12.837838
 Breathitt County        9.737828
 Breckinridge County    10.757840
 Bullitt County          2.385108
 Butler County           9.831461
 Caldwell County        12.941644
 Calloway County         6.747066
 Campbell County         4.207758
 Carlisle County        11.868687
 Carroll County         11.903853
 Carter County           7.414171
 Casey County           12.956143
 Christian County        5.515832
 Clark County            3.786342
 Clay County             9.306409
 Clinton County         15.075377
 Crittenden County      13.858040
 Cumberland County      16.812439
 Davies

In [81]:
rate_of_vacancy = sf1['pct_vacant'].groupby(sf1['county'])
rate_of_vacancy.mean()

county
 Adair County           15.868316
 Allen County           15.808521
 Anderson County         8.927276
 Ballard County         12.640382
 Barren County          11.444842
 Bath County            14.852074
 Bell County            10.308518
 Boone County            6.499883
 Bourbon County         11.112101
 Boyd County            10.097739
 Boyle County           10.332057
 Bracken County         13.577093
 Breathitt County       11.969659
 Breckinridge County    24.982799
 Bullitt County          5.651071
 Butler County          15.507969
 Caldwell County        14.784313
 Calloway County        14.392385
 Campbell County         9.502310
 Carlisle County        13.221629
 Carroll County         14.217337
 Carter County          13.412155
 Casey County           16.340594
 Christian County       16.249693
 Clark County            9.307550
 Clay County            13.976748
 Clinton County         18.174491
 Crittenden County      18.722653
 Cumberland County      22.529994
 Davies

In [82]:
rate_of_vacancy = sf1['pct_vacant'].groupby(sf1['county'])
rate_of_vacancy.quantile(0.9)

county
 Adair County           20.649714
 Allen County           22.324254
 Anderson County        12.075984
 Ballard County         14.051713
 Barren County          15.985510
 Bath County            15.791138
 Bell County            12.192074
 Boone County            9.911678
 Bourbon County         15.100240
 Boyd County            15.934394
 Boyle County           13.402266
 Bracken County         13.979335
 Breathitt County       13.997579
 Breckinridge County    50.773256
 Bullitt County          8.225072
 Butler County          21.444985
 Caldwell County        16.371811
 Calloway County        25.560676
 Campbell County        15.037431
 Carlisle County        14.436130
 Carroll County         16.653610
 Carter County          17.041743
 Casey County           19.082119
 Christian County       18.554342
 Clark County           12.037881
 Clay County            19.639306
 Clinton County         20.276716
 Crittenden County      23.443167
 Cumberland County      27.104038
 Davies

## Some review:

I've included in the data folder a shapefile with the Census geographies.  Can you use it to: 

1. Calculate the population density of each Census tract and county in Kentucky. 
2. Make a choropleth showing the population density.  

In [84]:
import geopandas as gp
from shapely.geometry import Point
census_data = gp.read_file('data/gz_2010_21_140_00_500k.shp')
census_data  #reading shp file

,GEO_ID,STATE,COUNTY,TRACT,NAME,LSAD,CENSUSAREA,geometry
0,1400000US21035010700,21,035,010700,107,Tract,74.333,"POLYGON ((-88.43010642581829 36.500896307, -88..."
1,1400000US21037050100,21,037,050100,501,Tract,0.205,"POLYGON ((-84.50608223252529 39.0950807457157,..."
2,1400000US21037050400,21,037,050400,504,Tract,0.186,"POLYGON ((-84.483948 39.09684, -84.48150699999..."
3,1400000US21037052100,21,037,052100,521,Tract,0.433,"POLYGON ((-84.4935547013056 39.1027204798606, ..."
4,1400000US21037052200,21,037,052200,522,Tract,0.452,"POLYGON ((-84.46285999999999 39.099847, -84.46..."
5,1400000US21043960300,21,043,960300,9603,Tract,70.201,"POLYGON ((-83.147093 38.303398, -83.1455429999..."
6,1400000US21045950500,21,045,950500,9505,Tract,80.060,"POLYGON ((-84.901112 37.116297, -84.916557 37...."
7,1400000US21047200200,21,047,200200,2002,Tract,1.743,"POLYGON ((-87.483228 36.875998, -87.483262 36...."
8,1400000US21047200700,21,047,200700,2007,Tract,3.613,"POLYGON ((-87.53568799999999 36.870476, -87.51..."
9,1400000US21047201501,21,047,201501,2015.01,Tract,0.772,POLYGON ((-87.44485399999999 36.66238999999999...


In [85]:
#Population Density
PD1=census_data['COUNTY'].groupby(sf1['county']).count()
PD1

county
 Adair County            7
 Allen County            6
 Anderson County         5
 Ballard County          3
 Barren County          10
 Bath County             3
 Bell County             9
 Boone County           22
 Bourbon County          6
 Boyd County            13
 Boyle County            7
 Bracken County          3
 Breathitt County        7
 Breckinridge County     6
 Bullitt County         18
 Butler County           5
 Caldwell County         3
 Calloway County         9
 Campbell County        25
 Carlisle County         3
 Carroll County          3
 Carter County           7
 Casey County            5
 Christian County       19
 Clark County           10
 Clay County             6
 Clinton County          3
 Crittenden County       4
 Cumberland County       2
 Daviess County         23
                        ..
 Nicholas County         2
 Ohio County             7
 Oldham County          14
 Owen County             3
 Owsley County           2
 Pendleton County    

In [86]:
PD2=census_data['CENSUSAREA'].groupby(sf1['county']).sum()
PD2

county
 Adair County            225.870
 Allen County            221.214
 Anderson County          99.404
 Ballard County          234.818
 Barren County             6.897
 Bath County               2.621
 Bell County             177.779
 Boone County            838.075
 Bourbon County          290.367
 Boyd County               4.829
 Boyle County             11.597
 Bracken County            2.851
 Breathitt County          9.729
 Breckinridge County      10.697
 Bullitt County          390.498
 Butler County           373.648
 Caldwell County           4.731
 Calloway County         336.910
 Campbell County        1443.507
 Carlisle County          39.506
 Carroll County          103.147
 Carter County           472.420
 Casey County            240.535
 Christian County        408.507
 Clark County            607.545
 Clay County             378.616
 Clinton County           21.600
 Crittenden County        23.857
 Cumberland County        58.178
 Daviess County         1091.321
   